In [1]:
import datetime as dt

from seabeepy.config import SETTINGS
from pathlib import Path

import seabeepy as sb
import requests
import json

In [2]:
def get_datasets(url) -> list[dict]:
    resp = requests.get(url).json()
    datasets = resp["datasets"]
    if resp["links"]["next"] is not None:
        datasets.extend(get_datasets(resp["links"]["next"]))

    return datasets

In [3]:
geonode_url = "https://geonode.seabee.sigma2.no"
datasets = get_datasets(f"{geonode_url}/api/v2/datasets")

In [4]:
datasets = [ds for ds in datasets if ds["subtype"] == "raster" and "niva" in ds["abstract"].lower()]
len(datasets)

95

In [5]:
dataset_no_year = [ds for ds in datasets if ds["subtype"] == "raster" and "niva-tidy/niva" in ds["abstract"].lower()]

In [6]:
len(dataset_no_year)

44

In [7]:
minio_path_lookup = {}
for ds in dataset_no_year:
    for s in ds["raw_abstract"].split(" "):
        if s.startswith("niva-tidy"):
            minio_path_lookup[ds['name']] = s.replace("niva-tidy", "niva-tidy/2023")

In [8]:
for layer_name, minio_path in minio_path_lookup.items():
    path = Path(f"/home/notebook/shared-seabee-ns9879k/{minio_path}")
    if path.exists() and (path / "orthophoto"/ f"{layer_name}.tif").exists():
        print(f"Updating {layer_name}")
        abstract = sb.geo.get_html_abstract(str(path))
        date = sb.ortho.parse_mission_data(path, parse_date=True)[2]
        metadata = {
            "abstract": abstract,
            "date": date.isoformat(),
            "date_type": "creation",
            "attribution": "SeaBee",
        }
        sb.geo.update_geonode_metadata(
            layer_name,
            SETTINGS.GEONODE_USER,
            SETTINGS.GEONODE_PASSWORD,
            metadata,
        )

Updating Mapping_larvik_olberg-all_202305051023_msi_100m
Updating Mapping_larvik_olberg-all_202309131050_rgb_100m
Updating Mapping_halden_h28_202308291429_rgb_80m
Updating Mapping_halden_h24_202308291354_rgb_80m
Updating Mapping_halden_h23_202308301032_rgb_80m
Updating Mapping_larvik_olberg-all_202309131140_msi_60m
Updating Mapping_halden_h20_202308301136_msi_80m
Updating Mapping_halden_h28_202308291436_msi_80m
Updating Mapping_halden_h34_202308300910_msi_80m
Updating Mapping_halden_h34_202308300859_rgb_80m
Updating Mapping_halden_h33_202308300943_rgb_80m
Updating Mapping_halden_h22_202308300752_msi_80m
Updating Mapping_halden_h35_202308300813_rgb_80m
Updating Mapping_halden_h24_202308291402_msi_80m
Updating Mapping_larvik_olberg-all_202309131050_msi_100m
Updating Mapping_larvik_olberg-all_202308161129_RGB_60m
Updating Mapping_larvik_olberg-s_202308160955_RGB_20m
Updating Mapping_larvik_olberg-all_202307140818_RGB_100m
Updating Mapping_larvik_olberg-all_202307140902_MSI_100m
Updating M

In [9]:
datasets_seabee_data = [ds for ds in datasets if ds["subtype"] == "raster" and "SEABEE_DATA" in ds["abstract"]]

In [10]:
len(datasets_seabee_data)

11

In [11]:
for ds in datasets_seabee_data:
    print(ds["title"])

Mapping_finnmark_vassbukta_202309281040_msi_100m
Mapping_halden_h22_202308300741_rgb_80m
Mapping_halden_h21_202308301118_msi_80m
Mapping_larvik_olberg-all_202308160923_MSI_100m
Mapping_larvik_olberg-all_202306231147_MSI_100m
Mapping_larvik_olberg-s_202306231037_MSI_40m
Mapping_oldberg_202306231122_MSI_40m
Mapping_halden_h25_202308291309_msi_80m
Mapping_halden_h29_202308290947_rgb_80m
Mapping_halden_h31_202308291138_msi_80m
Mapping_halden_h36_202308290907_rgb_70m


In [16]:
minio_path_lookup_seabee = {}
for ds in datasets_seabee_data:
    for s in ds["raw_abstract"].split(" "):
        if s.startswith("niva-tidy"):
            minio_path_lookup_seabee[ds['name']] = s.replace("SEABEE_DATA/", "")

In [19]:
for layer_name, minio_path in minio_path_lookup_seabee.items():
    path = Path(f"/home/notebook/shared-seabee-ns9879k/{minio_path}")
    if path.exists() and (path / "orthophoto"/ f"{layer_name}.tif").exists():
        print(f"Updating {layer_name}")
        abstract = sb.geo.get_html_abstract(str(path))
        date = sb.ortho.parse_mission_data(path, parse_date=True)[2]
        metadata = {
            "abstract": abstract,
            "date": date.isoformat(),
            "date_type": "creation",
            "attribution": "SeaBee",
        }
        sb.geo.update_geonode_metadata(
            layer_name,
            SETTINGS.GEONODE_USER,
            SETTINGS.GEONODE_PASSWORD,
            metadata,
        )

Updating Mapping_halden_h22_202308300741_rgb_80m
Updating Mapping_halden_h21_202308301118_msi_80m
Updating Mapping_larvik_olberg-all_202308160923_MSI_100m
Updating Mapping_larvik_olberg-all_202306231147_MSI_100m
Updating Mapping_larvik_olberg-s_202306231037_MSI_40m
Updating Mapping_oldberg_202306231122_MSI_40m
Updating Mapping_halden_h25_202308291309_msi_80m
Updating Mapping_halden_h29_202308290947_rgb_80m
Updating Mapping_halden_h31_202308291138_msi_80m
Updating Mapping_halden_h36_202308290907_rgb_70m
